# Package Managers - Yum

Installing Software using Package Managers

## Step 1
What is a package manager?
Package managers are programs which install, update, and remove software. Doing these operations by hand would be tedious and prone to mistakes. Bundling these functionalities into a single tool gives users a more streamlined experience.

YUM has even more utility because it performs dependency resolution whenever it installs, removes, or updates software. The upcoming steps will give examples of how to use YUM to streamline your package management.

Installing a package with yum
Install the network analysis application Wireshark using yum. This application is useful for troubleshooting network packets that are traveling in and out of your system. This functionality in particular is not important to this lab, this application was picked simply because it serves as a good test subject for the upcoming examples.


```bash
yum install -y wireshark
```

Note: The -y option will automatically answer yes to any prompts during the installation. Using this option is not best practice when installing software on your own system, but it is useful for streamlining some operations in this lab.

This installation generates a large output, so here is a high level summary of what each section means. The first section shows all of the packages that must be installed as dependencies for Wireshark:

```bash
<< OUTPUT ABRIDGED >>
Arch   Version         Repository                        Size
===================================================================
Installing:
wireshark
x86_64 1:2.6.2-12.el8  rhel-8-for-x86_64-appstream-rpms 3.7 M
Installing dependencies:
flac-libs
<< OUTPUT ABRIDGED >>
The next section of the output shows a verification status for each dependent package. In this case, YUM made sure that 36 other packages were up to date.

<< OUTPUT ABRIDGED >>
Verifying        : libgudev-232-4.el8.x86_64               35/36
Verifying        : pcre2-utf16-10.32-2.el8.x86_64          36/36
Installed products updated.
<< OUTPUT ABRIDGED >>
The final section of the output is a list of all of the packages that were installed in support of Wireshark's installation.

<< OUTPUT ABRIDGED >>
Installed:
  flac-libs-1.3.2-9.el8.x86_64                                     
  glx-utils-8.4.0-5.20181118git1830dcb.el8.x86_64                  
  gsm-1.0.17-5.el8.x86_64  
<< OUTPUT ABRIDGED >>
```

Complete!
You can verify this installation with the yum list command:

```bash
yum list wireshark

<< OUTPUT ABRIDGED >>
Installed Packages
wireshark.x86_64
         1:2.6.2-12.el8 @rhel-8-for-x86_64-appstream-rpms
```

This shows how powerful YUM is, as a single command will take care of the installation seamlessly. Next you will see how to update and remove packages with yum.

## Step 2
Updating a package
The list subcommand is useful for finding out which packages have available updates on the system.

```bash
yum list updates

<< OUTPUT ABRIDGED >>
Available Upgrades
bash.x86_64              4.4.20-1.el8_4                             rhel-8-for-x86_64-baseos-rpms
bind-export-libs.x86_64  32:9.11.26-4.el8_4                         rhel-8-for-x86_64-baseos-rpms
buildah.x86_64           1.19.7-2.module+el8.4.0+11311+9da8acfb     rhel-8-for-x86_64-appstream-rpms
<< OUTPUT ABRIDGED >>
```

Many packages on this system have updates available. Specifying a package name with yum update will constrain this operation to only look for that package.

NOTE: If you instead want to make sure your entire system is up to date, running yum update without any other arguments apply updates for all packages on your system (including YUM itself).

For this example, just update the bash package. 

```bash
yum update -y bash


<< OUTPUT ABRIDGED >>
===============================================================
 Package                Architecture       Size
===============================================================
Upgrading:
 bash                   x86_64             1.5 M

Transaction Summary
===============================================================
Upgrade  1 Package

<< OUTPUT ABRIDGED >>
```

NOTE: You sometimes see the subcommand upgrade being used somewhat interchangeably with update. The difference between these two subcommands is that upgrade will remove any obsolete packages from the system. Often the configuration for YUM is such that these subcommands will both carry out the upgrade operation.

Removing a package
Removing a package follows the same theme of simplicity.

```bash
yum remove -y httpd
```

The extensive output shows you information about which dependent RPMs were removed as part of this transaction.

```bash
<< OUTPUT ABRIDGED >>
Removing:
 httpd            x86_64 2.4.37-39.module+el8.4.0+9658+b87b2deb

Removing dependent packages:
 mod_ssl          x86_64 1:2.4.37-39.module+el8.4.0+9658+b87b2deb

Removing unused dependencies:
 apr              x86_64 1.6.3-11.el8

<< OUTPUT ABRIDGED >>

Removed:
  httpd-2.4.37-39.module+el8.4.0+9658+b87b2deb.x86_64               
  httpd-filesystem-2.4.37-39.module+el8.4.0+9658+b87b2deb.noarch    
  httpd-tools-2.4.37-39.module+el8.4.0+9658+b87b2deb.x86_64  

Complete!
```

Use the list subcommand to confirm that the package has been uninstalled:

```bash
yum list httpd

<< OUTPUT ABRIDGED >>
Available Packages
httpd.x86_64 2.4.37-39.module+el8.4.0+9658+b87b2deb
```

The package is now listed as Available rather than Installed. The next step will walk you through yum history which provides some more powerful tools for rolling back package transactions.

## Step 3
Using yum history to roll back a transaction
The history subcommand provides a summary of recent yum transactions.

```bash
yum history
```

The output contains transaction IDs in the first column, which are how you reference specific locations in the transaction history when executing rollbacks.

```bash

ID| Command                  | Date and time    | Action(s)| Altered
------------------------------------------------------------------
7 | remove -y httpd          | 2021-06-11 18:42 | Removed  | 3
6 | update -y bash           | 2021-06-11 18:41 | Upgrade  | 1   
5 | install -y wireshark     | 2021-06-11 18:40 | Install  | 36   
4 | install -y gcc llvm-libs | 2021-03-11 22:22 | Install  | 13   
3 | install -y buildah       | 2021-03-11 22:21 | I, U     | 216   
2 | install -y rsync         | 2021-03-11 22:20 | Install  | 1   
1 |     
```

This subcommand is also useful for rolling back transactions if you erroneously install or update a package, as it cleans up all of the dependencies associated with the package. Rollback the state of your system to before you uninstalled httpd:

```bash
yum -y history rollback 6
```

NOTE: The number 6 here is used here to specify that the rollback the state of the system to how it was at transaction ID 6, or before you removed httpd. You can use other relative offsets, such as last-3, or you can use absolute transaction IDs. For example, yum history rollback 2 would rollback to the transaction where rsync was installed.

```bash
<< OUTPUT ABRIDGED >>
Installed:
  apr-1.6.3-11.el8.x86_64                                           
  apr-util-1.6.1-6.el8.x86_64                                       
  apr-util-bdb-1.6.1-6.el8.x86_64                                   
  apr-util-openssl-1.6.1-6.el8.x86_64                               
  httpd-2.4.37-39.module+el8.4.0+9658+b87b2deb.x86_64               
  httpd-filesystem-2.4.37-39.module+el8.4.0+9658+b87b2deb.noarch          

Complete!
```
This command reinstalled the RPMs that are part of httpd (including all dependencies), restoring the system state to how it was before the previous transaction.

There are many more subcommands that you can use to customize how YUM behaves. Check out this [YUM Command Cheat Sheet for RHEL](rh_yum_cheatsheet_1214_jcs_print-1.pdf) for more info.

## Step 4
Using RPM to run detailed queries
YUM is the standard package manager, but this does not mean that RPM is obsolete. Detailed queries can be carried out using RPM using the query option (-q). This step will walk you through three examples of how to use this option.

Listing files installed by a package
You can combine the -q option with the list option (-l) to display all of the files that are installed by a specific package. This information is hidden by default to keep the package installation process simple, but it is often useful to know where specific files are located.
```bash
rpm -ql wireshark

/usr/bin/wireshark
/usr/lib/.build-id
/usr/lib/.build-id/9b
/usr/lib/.build-id/9b/f3ec88dd7a948dfaf1382498091fa57333c153
<< OUTPUT ABRIDGED >>
```

Now you can see that the Wireshark application is located at usr/bin/wireshark.

Listing installation scripts
Some packages run scripts to finish up their installation process. The --scripts option can be used in combination with rpm -q to return the contents of these postinstall scripts for a specific package. Wireshark does not actually have any postinstall scripts, so instead try this command with the httpd package.

```bash
rpm -q --scripts httpd

postinstall scriptlet (using /bin/sh):

if [ $1 -eq 1 ] ; then
        # Initial installation
        systemctl --no-reload preset httpd.service htcacheclean.service httpd.socket &>/dev/null || :
fi
<< OUTPUT ABRIDGED >>
```

The output shows a script that creates a preset for the httpd service behind the scenes when the package is installed.

Showing changelog
Specifying --changelog with rpm -q will display what has recently been changed in a package.

```bash
rpm -q --changelog wireshark

Tue Oct 29 2019 Michal Ruprich  - 1:2.6.2-12
- Resolves: #1609737 - wiretap does not write "drop_count" in pcapng format

* Mon Apr 29 2019 Michal Ruprich  - 1:2.6.2-11
- Related: #1602731 - Fixing multilib problem in devel subpackage
<< OUTPUT ABRIDGED >>
```

The next step will walk you through another use for rpm, file validation.

## Step 5
Validating package files using rpm
If a program is suddenly misbehaving, it could be related to some key files not being installed properly. rpm has a validation option (-V) which will compare the local files with the expected files for the package. Remove a file from the Wireshark package to test out this validation functionality:

```bash
rm -f /usr/share/icons/hicolor/256x256/apps/wireshark.png

Use the -V option with rpm to check the file status of Wireshark.

rpm -V wireshark

missing     /usr/share/icons/hicolor/256x256/apps/wireshark.png
Now that you have seen there is a missing file, reinstall the Wireshark package to return it to its proper state:

yum -y reinstall wireshark

<< OUTPUT ABRIDGED >>
Arch   Version        Repository                        Size
===================================================================
Reinstalling:
wireshark
x86_64 1:2.6.2-12.el8 rhel-8-for-x86_64-appstream-rpms 3.7 M
<< OUTPUT ABRIDGED >>
Check that wireshark.png has been restored:

ls /usr/share/icons/hicolor/256x256/apps/

fedora-logo-icon.png  wireshark.png
```

The command rpm -V is useful for more than just finding missing files. This command can inform the user if any files have changed since installation. To test this out, run the validation command on the PAM package.

```bash
rpm -V pam

S.5....T.  c /etc/pam.d/password-auth
S.5....T.  c /etc/pam.d/system-auth
```

This query returns two files with unexpected criteria, password-auth and system-auth. The first column of this output tells you that both of these files have an unexpected size, checksum, and timestamp. Both of these files are configuration files, so it makes sense that they would be modified after installation to suit the purpose of this particular system.

